**Make sure to do the extraction and other process and importing the schema, just read the fucking documentation**

I guess you won't do that so let me do that, **just upload that fucking zip**

**but I can't upload the schema so just upload the fucking schema**

In [ ]:
import shutil

# Path to your zip file
zip_path = "/content/sql_db_model_lora.zip"

# Extract the contents to the current directory
shutil.unpack_archive(zip_path, "/content/")

print("Unzipping complete! Files are extracted to /content/")

Unzipping complete! Files are extracted to /content/


In [ ]:
!pip install torch transformers peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 919.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import json

# 1. Path to your extracted LoRA adapters
lora_path = "./lora_adapters"  # Update this path

# 2. Load the base model and tokenizer
base_model_name = "defog/sqlcoder-7b-2"
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Load the base model in 4-bit to save memory
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    load_in_4bit=True,
    torch_dtype=torch.float16
)

# 3. Load the LoRA adapters onto the base model
model = PeftModel.from_pretrained(base_model, lora_path)
print("Model with LoRA adapters loaded successfully!")

# 4. Define the schema (needs to be the same as during training)
with open('database_schema.json', 'r') as f:
    db_schema_dict = json.load(f)

# Convert the schema dict to a CREATE TABLE format
schema_str = ""
for table_name, columns in db_schema_dict.items():
    schema_str += f"CREATE TABLE {table_name} (\n"
    column_definitions = []
    for col_name, col_type in columns.items():
        column_definitions.append(f"  {col_name} {col_type}")
    schema_str += ",\n".join(column_definitions)
    schema_str += "\n);\n\n"

# 5. Create an inference function
def generate_sql_query(natural_language_query):
    """Generate a SQL query from natural language using the fine-tuned model"""
    # Prepare the prompt
    prompt = f"""
### Instructions:
Convert the following natural language query into a SQL query.

### Database Schema:
{schema_str}

### Query:
{natural_language_query}

### SQL Query:
"""
    # Tokenize the prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Generate SQL query
    with torch.no_grad():
        outputs = model.generate(
            inputs.input_ids,
            max_length=1024,
            num_beams=5,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            top_k=50,
            repetition_penalty=1.2,
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode the output
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the SQL part (everything after the last "### SQL Query:")
    sql_parts = generated_text.split("### SQL Query:")
    if len(sql_parts) > 1:
        return sql_parts[-1].strip()
    return generated_text.strip()

# 6. Test the model
test_query = "Show me assignment data upto assignment5"
generated_sql = generate_sql_query(test_query)
print(f"Test Query: {test_query}")
print(f"Generated SQL: {generated_sql}")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Model loaded successfully with LoRA adapters!
Natural Language Query: Show me students who scored more than 80 in all assigments from assignment table
Generated SQL Query: SELECT s.name, AVG(a.score) AS average_score FROM student s JOIN assignment a ON s.id = a.student_id GROUP BY s.name HAVING AVG(a.score) > 80 ORDER BY average_score DESC NULLS LAST;
